In [1]:
import sys
CURRENT_DIR = r'C:\Users\Kart\jupyter_projects\finance\MyAccounting'
sys.path.insert(0, CURRENT_DIR)

from MyAccounting import MyAccounting
settings_path=r'C:\Users\Kart\jupyter_projects\finance'
acc= MyAccounting(settings_path)

driver loaded


In [2]:
acc.add_account('58','Финансовые вложения','Счет 58 «Финансовые вложения» предназначен для обобщения информации о наличии и движении инвестиций организации в государственные ценные бумаги, акции, облигации и иные ценные бумаги других организаций, уставные (складочные) капиталы других организаций, а также предоставленные другим организациям займы.')

Error: E11000 duplicate key error collection: exchange_operations.accounts_descr index: account_number_1 dup key: { : "58" }


In [3]:
acc.driver.get_data('accounts_info')

,account_descr_long,account_descr_short,account_id,account_number,account_parrent,creation_date
0,Счет 58 «Финансовые вложения» предназначен для...,Финансовые вложения,1,58,None,2019-02-03 10:11:03.298


In [ ]:
acc.driver.drop_db()

In [ ]:
acc.driver.get_data('documents')

In [ ]:
doc=document('test doc','2017-01-01')

In [ ]:
acc.driver.add_row('documents',doc)

In [ ]:
settings={'driver':'mongo',
          'db_name':'exchange_operations',
          'schema_name':'exchange_operations',
          'accounts_info':'accounts_descr',
          'accounts':'accounts',
          'documents':'documents',
          'host':None,'port':None,
          'transactions':'transactions',
          'variables':'variables'}

#acc.add_config (**settings)

In [ ]:
cl=MongoClient()
db=cl['finance']

In [ ]:
d.drop_db()

In [ ]:
class document():
    __slots__ ='document_descr','date'
    def __init__(self,document_descr :int,date):
        #self.document_id=document_id
        self.document_descr=document_descr
        self.date=date 

In [ ]:
doc=document('test doc','2017-01-01')

In [ ]:
d.add_row('documents',doc)

In [ ]:
cl=MongoClient()
test_db=cl['exchange_operations']

In [ ]:
import pandas as pd

pd.DataFrame(d.get_data('documents'))

In [ ]:
import pprint
col=test_db['documents']
for post in col.find():
    pprint.pprint(post)

In [ ]:
d.variables.update_one({'_id': k}, {'$inc': {'var': 3}})

In [ ]:
dict_trn={'document_id':0,'account_id':0,'document_id':0,'trans_id':0}
d.variables.insert_one({'check3':11,'value':dict_trn}).inserted_id

In [ ]:
from pymongo import MongoClient,DESCENDING,ASCENDING
acc_engine=MyAccounting()
acc_engine.db_client = MongoClient()   

In [ ]:
import datetime

datetime.datetime.now(),datetime.datetime.now().fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
from os import path,makedirs
import json
from MyAccounting.attributes import config,account,document,transaction,accounts_info

class MyAccounting():
    __slots__ ='config_path','config_empty','config','config_filename','driver'
    
    def __init__(self, config_path:str ):
        self.config_filename='MyAccounting_settings.json'
        self.config_path= config_path     
        file_path,path_exists=self.__check_file_existance(config_path)      
        if path_exists:
            self.config_empty=False  
            self.config_filename=file_path
            self.__load_config()
            self.__load_driver()
        else:
            self.config_empty=True
            print('Config file requed:use add_config function')
            
    def __check_file_existance(self,config_path: str)-> (str,bool):
        file_path=path.join(config_path,self.config_filename)
        return file_path,path.exists(file_path)
 
    def __load_config(self):
        try:
            with open(self.config_filename) as f:
                data = json.load(f)
            self.config=data
        except:
            print('Error reading config file')
            
    def __load_driver(self):
        if self.config['driver']=='mongo':
            from MyAccounting.drivers import MongoDriver
            self.driver=MongoDriver(self.config)
            print ("driver loaded")
        else:
            print('Unknown driver')
            

    def add_config(self,**kwargs):

        if self.config_empty:
            if not path.exists(self.config_path):
                makedirs(self.config_path)
            filename=path.join(self.config_path,self.config_filename)   
            self.config_filename=filename
            
            conf=config(**kwargs)

            with open(filename,'w') as f:
                json.dump(conf.data, f)
            self.config_empty=False
            self.__load_config()

    def add_account(self,account_number :str,account_descr_short :str,account_descr_long :str,account_parrent:int =None):
        accounts_info(account_number,account_descr_short,account_descr_long,account_parrent)

In [ ]:
    def add_account(self,account_number :str,account_descr_short :str,account_descr_long :str,account_parrent:int =None):
        acc=accounts_info(account_number,account_descr_short,account_descr_long,account_parrent)
        self.driver.add_row('accounts_info',acc)

In [ ]:
try:
    a=1/0
except AssertionError as error:
    print(error)


In [ ]:
    def add_row(self,operation_type,obj,creation_date=None)->str:
        res={key:getattr(obj, key) for key in obj.__slots__}
        attr=self.mapping[operation_type][0]
        return_message=(True,'Ok')

        if attr is not None:
            res[attr]=self.__increment_id(attr)
            
        if creation_date is None:
            res['creation_date']=datetime.datetime.now()
        else:
            res['creation_date']=creation_date
        try:
            getattr(self,operation_type).insert_one(res)
        except Exception as e: 
            return_message=(False,e)
        return return_message